This script contains cypher chunks of code used for the TikTok graph analysis. 
All the produced graphs are later exported to Gephi to be further visualized.
The cypher code is not adapted to be run in Python. Instead, paste the chunks directly into neo4j.

Exporting users talking about Ukraine and Russia, such that we get a deep look into the network:

In [ ]:
call apoc.export.graphml.query("MATCH (a:user)-[r1:recommends]-(b:user), (a:user)--(v:video)--(t:tag), (b:user)--(w:video)--(s:tag) 
                                WHERE (v.desc=~".*putin.*" OR v.desc=~".*zelensky.*") AND 
                                      (w.desc=~".*putin.*" OR w.desc=~".*zelensky.*") AND 
                                      v.commentCount > 1000 AND  t.title IN ['ukraine','russia','ukrainewar', 'ukrainerussiaconflict', 'ukrainewarrussia', 'supportukraine', 'ukrainevsrussia', 'prayforukraine', 'russianarmy'] and 
                                      s.title IN ['ukraine','russia','ukrainewar', 'ukrainerussiaconflict', 'ukrainewarrussia', 'supportukraine', 'ukrainevsrussia', 'prayforukraine', 'russianarmy']
                                RETURN a, b, r1 ", 
                               "usersTalkingUkraine.graphml", 
                               {useTypes: true, storeNodeIds:true, readLabels:true})

Calculating the shortest path between #ukrainewar and #kardashians:

In [ ]:
MATCH (t1:tag {title: 'ukrainewar'}), (t2:tag {title: 'kardashians'}),
      p = shortestPath((t1)-[:has*]-(t2))
RETURN p

Adding tag occurence frequencies as a tag node property:

In [ ]:
MATCH (:user)--(:video)--(t:tag) 
WITH t, count(t) AS freq 
SET t.freq = freq 
RETURN t.title, freq ORDER BY freq desc LIMIT 100 

Creating tag_connected relationship, so we can extract tags that are connected to #ukraine:

In [ ]:
MATCH (a:tag)-[:has*1..2]-(b:tag) 
WHERE a.title = 'ukraine'
MERGE (a)-[e:tag_connected_ukraine]-(b) 
RETURN a, e, b LIMIT 50

Tags connected to #ukraine with frequency greater that 100:

In [ ]:
MATCH (a:tag)-[e:tag_connected_ukraine]-(b:tag) 
WHERE b.freq > 100 
RETURN a, e, b LIMIT 200

We also ran Page Rank in cypher, but ended up not using it in the further analysis. 

In [ ]:
// Projecting ukraineTags
call gds.graph.project.cypher('ukraineTags',
                            "MATCH (u:user)-[]->(:video)-[]->(t:tag) 
                            WHERE t.title IN ['ukraine', 'russia', 'ukrainewar', 'ukrainerussiaconflict', 'ukrainewarrussia', 'supportukraine', 'ukrainevsrussia', 'prayforukraine', 'russianarmy'] 
                            RETURN id(u) AS id",
                            "MATCH (u1:user)-[:recommends]->(u2:user) 
                            RETURN id(u1) as source, id(u2) as target, count(*) as weight",
                            {validateRelationships:FALSE})

In [ ]:
// PageRank on ukraineTags
CALL gds.pageRank.stream('ukraineTags')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId) AS name, score
ORDER BY score DESC, name ASC

In [ ]:
// Write pageRank result as a node property
CALL gds.pageRank.write('ukraineTags', {writeProperty: 'pageRank'})